In [160]:
from flask import Flask, render_template, request, redirect, url_for, session
import duckdb
import pandas as pd
import os

In [161]:
app = Flask(__name__)

In [162]:
app.config['SECRET_KEY'] = 'your_very_secret_key'

In [163]:
BASE_DIR = "C:/Users/r0352/Desktop/DBMS_Final_Web"
csv_user_transations_path = os.path.join(BASE_DIR, 'dataset', 'user_transations.csv')
csv_user_card_data_path = os.path.join(BASE_DIR, 'dataset', 'user_card_data.csv')
csv_mcc_with_categories_path = os.path.join(BASE_DIR, 'dataset', 'mcc_with_categories.csv')

csv_user_spending_capability_path = os.path.join(BASE_DIR, 'dataset', 'user_spending_capability.csv')
csv_merchant_table_path = os.path.join(BASE_DIR, 'dataset', 'merchant_table.csv')


In [164]:
conn = duckdb.connect()

sql_query = f"SELECT * FROM read_csv_auto('{csv_merchant_table_path}')"
result = conn.sql(sql_query).fetchdf()
print(result)
print(type(result))


     merchant_id     merchant_city merchant_state    zip   mcc
0          59935            Beulah             ND  58523  5499
1          67570        Bettendorf             IA  52722  5311
2          27092             Vista             CA  92084  4829
3          27092       Crown Point             IN  46307  4829
4          13051           Harwood             MD  20776  5813
..           ...               ...            ...    ...   ...
995        81833      Hebbronville             TX  78361  5912
996        22204        Germantown             MD  20874  5541
997        50783           Jarreau             LA  70749  5411
998        71774  Clinton Township             MI  48036  5921
999        32175      Saint Peters             MO  63376  7538

[1000 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>


In [165]:
@app.route("/")
def home_page():
    return render_template("home-page.html")


In [166]:
@app.route("/user_spend")
def user_spending_page():
    return render_template("user-spend-page.html",title='User Spending Capability')
    

In [167]:
@app.route("/user_spend_input", methods=['POST'])
def user_spend_input():
    user_spend_input = int(request.form.get('client_id'))

    session['user_spend_input'] = user_spend_input
    return render_template("user-spend-page.html",title='User Spending Capability')
    

In [168]:
@app.route("/user_spend_output")
def user_spend_output():
    user_spend_input = session.pop('user_spend_input', [])
    conn = duckdb.connect()
    
    sql_query = f"SELECT * FROM read_csv_auto('{csv_user_spending_capability_path}') WHERE Client_ID={user_spend_input}"
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='User Spending Capability')
    

In [169]:
@app.route("/merchant_analysis")
def merchant_analysis_page():
    return render_template("merchant-analysis-page.html",title='Merchant Table')
    

In [170]:
@app.route("/merchant_analysis_input", methods=['POST'])
def merchant_analysis_input():
    merchant_analysis_input = int(request.form.get('merchant_id'))

    session['merchant_analysis_input'] = merchant_analysis_input
    return render_template("merchant-analysis-page.html",title='Merchant Table')
    

In [171]:
@app.route("/merchant_analysis_output")
def merchant_analysis_output():
    merchant_analysis_input = session.pop('merchant_analysis_input', [])
    conn = duckdb.connect()
    
    sql_query = f"SELECT * FROM read_csv_auto('{csv_merchant_table_path}') WHERE merchant_id={merchant_analysis_input}"
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='Merchant Table')
    

In [172]:
@app.route("/user_update")
def user_update_page():
    return render_template("user-update-page.html")

In [173]:
@app.route("/user_card")
def user_card():
    return render_template('user-card-page.html', title='User Card Data')

In [174]:
@app.route("/user_card_input", methods=['POST'])
def user_card_input():
    user_card_input = int(request.form.get('card_id'))

    session['user_card_input'] = user_card_input
    return render_template('user-card-page.html', title='User Card Data')

In [175]:
@app.route("/user_card_output")
def user_card_output():
    user_card_input = session.pop('user_card_input', [])
    conn = duckdb.connect()
    
    sql_query = f"SELECT * FROM read_csv_auto('{csv_user_card_data_path}') WHERE card_id={user_card_input}"
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='User Card Data')

In [176]:
@app.route("/user2")
def user2():
    return render_template('query-sample-2-page.html', title='SQL Query Sample 2')

In [177]:
@app.route("/user2_input", methods=['POST'])
def user2_input():
    user2_input = int(request.form.get('client_id'))

    session['user2_input'] = user2_input
    return render_template('query-sample-2-page.html', title='SQL Query Sample 2')

In [178]:
@app.route("/user2_output")
def user2_output():
    user2_input = session.pop('user2_input', [])
    conn = duckdb.connect()
    
    sql_query = f"""
    SELECT m.Category, SUM(t.amount) AS total_spend
    FROM read_csv_auto('{csv_user_transations_path}') AS t
    JOIN read_csv_auto('{csv_user_card_data_path}') AS c ON t.card_id = c.card_id
    JOIN read_csv_auto('{csv_mcc_with_categories_path}') AS m ON t.mcc = m.MCC_code
    WHERE c.client_id = {user2_input}
    GROUP BY m.Category
    ORDER BY total_spend DESC;
    """ 
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='SQL Query Sample 2')

In [ ]:
if __name__ == '__main__':
    app.run( )

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Nov/2025 15:08:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:08:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Nov/2025 15:08:31] "GET /user2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:08:37] "POST /user2_input HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:08:38] "GET /user2_output HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:08:41] "GET /user_card HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:08:57] "POST /user_card_input HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:08:58] "GET /user_card_output HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:09:01] "GET /merchant_analysis HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:09:15] "POST /merchant_analysis_input HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:09:17] "GET /merchant_analysis_output HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:09:19] "GET /user_spend HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2025 15:09:46] "POST /user_spend_input HTTP/1